### BERT prompt-to-playlist generator (using FAISS)

We use BERT to encode tracks as embeddings, and our prompt as well. Using FAISS similarity scores, we create a new playlist with k tracks which should please the user.

https://faiss.ai/index.html

In [1]:
import pandas as pd 
import numpy as np 
import faiss 
from sentence_transformers import SentenceTransformer 
from tqdm import tqdm 

Prompt input (ex. "UK alternative rock 80s") and size of created playlist (ex. 30)

In [2]:
#prompt = "UK alternative rock 80s"
prompts = [
    "UK alternative rock 80s", 
    "somber trap from 2010", 
    "nostalgic feminine 2000s pop", 
    "chill acoustic music to study", 
    "songs to scream to", #i asked a friend for prompts lol
    "2015 k-pop hits" 
]

playlist_size = 30

features = [
    "popularity","year","genre","danceability","energy","key","loudness","mode",
    "speechiness","acousticness","instrumentalness","liveness","valence","tempo",
    "duration_ms","time_signature"
]

Importing tracks data

In [3]:
path = "./final_data.csv"
df = pd.read_csv(path) 

df = df.dropna(subset=["track_name_mpd", "artist_name_mpd"])  #deletes invalid tracks, we need a min of info
df["genre"] = df["genre"].fillna("unknown")
df["year"] = df["year"].fillna(-1).astype(int)

print(f'Loaded {len(df)} tracks')

C:\Users\Elio\AppData\Local\Temp\ipykernel_17644\1726114263.py:2: DtypeWarning: Columns (0,3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Loaded 155966 tracks


In [4]:
def safe(x, default="unknown"): 
    return x if pd.notna(x) else default 

def track_to_text(row): 
    """
    Converts each row (track) of the dataset into a rich natural language 
    descritption which BERT can use efficiently and embed meaningfully.
    """
    features_text = ", ".join(
        f"{col} {safe(row[col])}" for col in features if col in row
    )
    return (
        f"{safe(row['track_name_mpd'])} by {safe(row['artist_name_mpd'])}. "
        f"From album: {safe(row['album_name_mpd'])}. "
        f"Position in playlist: {safe(row['position'])}. "
        f"{features_text}. "
        f"From playlist: {safe(row['playlist_name'])}."
    )

df["text_rpz"] = df.apply(track_to_text, axis=1) 
texts = df["text_rpz"].tolist() 

Creating tracks embeddings with BERT

In [5]:
#Loading BERT model (17 mins)
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
#Encoding tracks
track_embeddings = model.encode(texts, normalize_embeddings=True, show_progress_bar=True)
track_embeddings = track_embeddings.astype(np.float32)
dim = track_embeddings.shape[1]
print(f"Track embeddings shape: {track_embeddings.shape}")

Batches:   0%|          | 0/4874 [00:00<?, ?it/s]

Track embeddings shape: (155966, 768)


Building FAISS index

In [6]:
index = faiss.IndexFlatIP(dim) #cosine similarity
index.add(track_embeddings)

Playlist embeddings

In [7]:
playlist_groups = df.groupby("playlist_id")
playlist_embeddings = {}
for pname, group in tqdm(playlist_groups, desc="Encoding playlist embeddings"):
    emb = model.encode(group["text_rpz"].tolist(), normalize_embeddings=True)
    playlist_embeddings[pname] = np.mean(emb, axis=0)  # mean of tracks

Encoding playlist embeddings: 100%|██████████| 5076/5076 [17:05<00:00,  4.95it/s]


Generating new playlist

In [8]:
def generate_playlist(prompt: str, k: int, alpha=0.7): 
    """
    Given a natural language prompt, return the k most similar tracks.

    alpha: weight btw pormpt similarity & playlist correlation 
    """
    prompt_embedding = model.encode(prompt, normalize_embeddings=True).astype(np.float32) 
    
    #faiss search
    D, I = index.search(prompt_embedding.reshape(1, -1), k*3)
    candidate_tracks = df.iloc[I[0]].copy()

    def correlation_score(track_row):
        score = 0.0
        for pid in df[df["track_id"]==track_row["track_id"]]["playlist_id"].unique():
            if pid in playlist_embeddings:
                score += np.dot(prompt_embedding, playlist_embeddings[pid])
        return score

    #playlist correlation score
    candidate_tracks["playlist_corr"] = candidate_tracks.apply(correlation_score, axis=1)

    #combined with prompt similarity/correlation score (smh)
    candidate_tracks["combined_score"] = alpha*1.0 + (1-alpha)*candidate_tracks["playlist_corr"]

    #sort & remove duplicates
    result = candidate_tracks.sort_values("combined_score", ascending=False)
    result = result.drop_duplicates(subset=["track_name_mpd","artist_name_mpd"]).head(k)

    return result.reset_index(drop=True)[[
        "playlist_name", "track_name_mpd","artist_name_mpd", "year", "genre", "album_name_mpd"
    ]]

Use case example: 

In [9]:
playlists = [] 

for prompt in prompts: 
    playlists.append( generate_playlist(prompt, playlist_size) )

print("\ngenerated playlists:")
for i, pl in enumerate(playlists):
    print(f"\nPrompt: {prompts[i]}")
    print(pl.to_string(index=False)) 


generated playlists:

Prompt: UK alternative rock 80s
                          playlist_name                 track_name_mpd      artist_name_mpd  year   genre                     album_name_mpd
                       made in the a.m.                        History        One Direction  2015     pop                   Made In The A.M.
                                  Angie                Impossible Year  Panic! At The Disco    -1 unknown                Death Of A Bachelor
                                   Punk            Where Have You Been   Falling In Reverse  2013   metal                   Fashionably Late
             80s Hits - Best of the 80s            The Final Countdown               Europe    -1 unknown                                NaN
             80s Hits - Best of the 80s                     My Sharona            The Knack    -1 unknown                                NaN
                               My music                        19-2000             Gorillaz  2001 h